In [1]:
import requests
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from textblob import TextBlob

## Get article links for each company through NewsAPI

In [2]:
response = requests.get("https://newsapi.org/v2/everything?q=microsoft&sources=reuters&pageSize=100&apiKey=04fa90910183440c97c0e04751bc72c4")
microsoft_news_json = response.json()
microsoft_art = microsoft_news_json['articles']


In [3]:
def get_time_links(json):
    dicts = {}
    for x in range(0,100):
        time = json['articles'][x]['publishedAt']
        url = json['articles'][x]['url']
        dicts.update( { time : url} )
    return dicts

In [4]:
microsoft_links = get_time_links(microsoft_news_json)

In [5]:
len(microsoft_links)

98

In [6]:
def clean_time(links):  
    empty = {}
    for key, value in links.items():
            datetime = key.split('T')
            time = datetime[1].strip('Z')
            datetime[1] = time
            datetime = ' '.join(datetime)
            empty.update({datetime : value})
    return empty


In [7]:
cleaned_micro_links = clean_time(microsoft_links)

## Scrape Article Content

In [8]:
def get_content(urls_list):
    article_contents = {}
    for key, value in urls_list.items():
        r = requests.get(value)
        html = r.text
        soup = BeautifulSoup(html)
        foundp = soup.findAll('p')
        
        content = str()
        for para in foundp:
            text = para.text
            string_text = str(text)
            content += string_text
        article_contents.update({key : content})
    return article_contents
#         article_contents.append(content)
#     return article_contents
        

In [18]:
microsoft_content = get_content(cleaned_micro_links)

In [23]:
list(microsoft_content.keys())[0]

'2019-01-30 21:18:36'

&nbsp;

## Get Sentiment each article

In [27]:
blob = TextBlob(list(microsoft_content.values())[0])

In [28]:
polarity = blob.sentiment.polarity, blob.sentiment.subjectivity

In [29]:
list(polarity)

[0.06890294312169314, 0.38037781084656075]

In [46]:
for key, value in microsoft_content.items():
    blob = TextBlob(value)
    polarity = blob.sentiment.polarity, blob.sentiment.subjectivity
    sentiment = list(polarity)
    print(sentiment)
    if sentiment[0] > .05:
        print(value)
        print(sentiment[0])

[0.06890294312169314, 0.38037781084656075]
4 Min Read(Reuters) - Microsoft Corp on Thursday met Wall Street’s targets for its quarterly results and forecast, though Azure cloud computing sales grew more slowly than a year earlier. Shares of Microsoft, one of the most valuable U.S. technology companies, fell 3 percent in extended trade even though earnings per share slightly beat analysts’ estimates. The stock had closed 3.3 percent higher in a broad tech rally. Azure, Microsoft’s flagship cloud product, had revenue growth of 76 percent in the fiscal second quarter ended Dec. 31, down from a 98 percent surge a year earlier. Azure sales increased 76 percent in the September quarter as well. Long known for its Windows software, Microsoft has shifted its focus to the fledgling cloud market where it is battling Amazon.com Inc for dominance. The company is rapidly picking up business from the retail industry in particular, which is aiming to keep pace with the e-commerce business of Amazon. 

[0.06422077922077922, 0.35585714285714287]
8 Min ReadTALLINN (Reuters) - Estonia’s push to become a digital society has left it vulnerable to dirty money and sanction breaches, the country’s top banking regulator has warned. The Baltic state took center stage in one of the largest-ever money laundering scandals last year - the Estonian branch of Danske Bank helped funnel money from Russia and other ex-Soviet states, a report by the Danish lender showed. Danske Bank is now being investigated in Denmark, Estonia, Britain and the United States over the 200 billion euros ($227 billion) of suspicious payments. The Danske debacle was rooted in old school subterfuge – offshore shell companies were used to disguise where the money was coming from. But Estonia’s population of e-residents, some of whom bank in the country even though they live abroad, potentially offers a high-tech route for suspect funds. “The lesson of Danske is, I hope, enough for us,” Kilvar Kessler, the head of Estonia’s fi

In [50]:
import mediacloud.api
mc = mediacloud.api.MediaCloud('9bdec5b06584ca67419e76e8d9cc3997e40f9184e14e1af7251fc5ee7b753672')
res = mc.storyCount('microsoft AND tags_id_media:58722749', 'publish_date:[NOW-1YEAR TO NOW]')
print(res['count'])

12370
